# Industry Accelerators - SME Banking Model Scoring

## Introduction

In this notebook we will package all the models deployed throughout the execution of the previous notebooks and deploy them in the form on a Python Function. Through the scoring against that deployed function, we will be able to get predictions out of all the models. The models we will be deploying through this Python Function are:

* acceptance_classifier: to predict if a loan is fully paid or charged off.
* loan_life_classifier: to predict if the life of a loan is more than a year or not.
* credit_limit_model: to predict the amount of loan given to a customer. This model incorporates the results of the first two models.

This notebook defines the function `optimal_credit_line` and deploys it. The function calls the three deployed models, takes  an applied loan amount from the user as input, and returns the results. The results include if the loan is accepted or rejected and in the case of acceptance, it suggests a loan amount.

The preview of the process is as follows:

1. Load the dependencies
2. Generate the configuration parameters
3. Define a Python function to be deployed
4. Test the function locally
5. Deploy the function
6. Test the deployed function

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020, 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

## I/O format

### Input values

Expects a JSON with the schema:

`
{
    "input_data": [
        {
            "fields": str[],
            "values": any[][],
        }
    ]
}
`

- fields (str[]): A list of variable names.
- values (any[][]): A list of list of values.
  

The 'fields' property must contains the following values:

- All the customer data that is used as input in the 'acceptance_classifier' in `1-model-training` notebook.

- loan_amnt_applied: The loan amount that is requested

The 'values' property is a list of lists of values. Each inner list contains a value for each of the field value (in the same order of course).


Here is an example of customer data fields and values:

{'input_data': [{'fields': ['loan_amnt', 'term', ...], 'values': [11000.0, 0.0, ...]}]}

### Output values

#### If successful

returns a JSON with the schema:

`
{
    "predictions": [
        {
            "success": True,
            "approved": bool,
            "amount": float
        }
    ]
}
`

- success (bool): Always set to True. Indicate that the operation was successful.
- approved (bool): Indicates if the loan has been approved.
- amount (float): The loan amount that has been approved.
   
#### If not successful

returns a JSON with the schema:

`
{
    "predictions": [
        {
            "success": False,
            "status_code": int,
            "message": str
        }
    ]
}
`
        
- success (bool): Always set to False. Indicate that the operation failed.
- status_code (int): The error code. Any runtime error will generate a 400 (bad request) response.
- message (str): The error message.
                

## Step 0: If you are running on Cloud Pak for Data as a Service (Public Cloud)
A pre-requisite of running the rest of the cells is to have:

- From the "More" menu above, select the "Insert project token" option
- Enter your "API Key" and "location" (e.g. 'us-south') in the cell below

In [ ]:
# If running on Cloud, enter your API key and instance location below.
api_key = ''
location = 'us-south'

## Load the dependencies

In [ ]:
import pandas as pd
import numpy as np
import pickle, os
import warnings
warnings.filterwarnings('ignore')

# When running on Cloud click the option "Insert project token" - if not running on cloud project is setup below
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project() 

## Set up the configuration parameters

The deployed function requires a few configuration parameters. These include:

- The ID of the deployment space in which it is deployed

- The ID of the three deployed models that it uses behind the scenes

- The credentials to connect to this deployment space


#### Connect to WML client
We will use the watson_machine_learning_client package to establish a connection to the Watson Machine Learning API with the system credentials.

In [ ]:
from ibm_watson_machine_learning import APIClient
import os

if api_key != '':
    wml_credentials = {
        "apikey": api_key,
        "url": 'https://' + location + '.ml.cloud.ibm.com',
        "version": "3.5"
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']

    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }

client = APIClient(wml_credentials)

## Find the target deployment space and connect to it

In [ ]:
# Specify a name for the target deployment space
SPACE_NAME = 'SMEBanking'

# Check is the space already exists
space_uid = ''
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == SPACE_NAME:
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

if space_uid == '':
    print("Deployment space",SPACE_NAME,"not found.")

In [ ]:
# List all Deployments
print(client.deployments.list())

#### User inputs

We need the IDs of the three models that are used to make the predictions, namely, the acceptance classifier, the loan life classifier, and the credit limit model.

In [ ]:
acceptance_classifier_id = "c952df33-28b1-411d-b536-52f3d7a0c584"
loan_life_classifier_id = "107cba9d-9257-4d89-bd1f-01b9cfce7c3e"
credit_limit_model_id = "4ce516ca-7fd2-4f14-a6e7-c098cf577e8a"

In [ ]:
params = { 
    "wml_credentials": wml_credentials,
    "space_uid": space_uid,
    "acceptance_uid": acceptance_classifier_id,
    "life_uid": loan_life_classifier_id,
    "credit_uid": credit_limit_model_id
}

## Define a Python function to be deployed

A Python function is a feature to save and deploy Python code through notebooks or IDE. Python functions can be implemented in Python notebooks or through REST API using IDE.
The requirement of a Python function is to have a score() function inside the Python function. The score() function will be called when running the deployed Python function.

A Python function can be:

- Saved in the deployment space repository.
- Deployed in the deployment space repository.
- Scored.

The function to be deployed takes the configuration parameters as a default input argument.

In [ ]:
def optimal_credit_line(params=params):
    
    # ---------------------------------------------------------
    # This part of the code runs only once, when the deployment
    # is started
    # ---------------------------------------------------------
    
    # Import dependencies
    import requests
        
    # Set global variables 
    approved_value = 0.0
    DEBUG = False
    round_to = 1000
    max_requested_amnt = 1000000
    min_requested_amnt = 1000
    
    def _format_error(code, message):
        """Return the JSON formatted error object.

        A requirement of CP4D is that any deployed function must return a JSON 
        serializable object that contains the property 'predictions'. Our returned
        object therefore nests the usual success flag, the status code, and the
        error message within this 'predictions' property.

        Args:
            code (int): The error code. 
            message (str): The error message.
            

        Returns:
            An object with one property 'predictions' that contains:
                success (bool): Always set to False. Indicate that the operation failed.
                status_code (int): The error code. Any runtime error will generate a 400 (bad request) response.
                message (str): The error message.
        """
        
        return {
            "predictions": [
                {
                    "success": False,
                    "status_code": code,
                    "message": message
                }
            ]
        }
        
    
    def _format_response(approved, amount=0):
        """Return the JSON formatted response object.

        A requirement of CP4D is that any deployed function must return a JSON 
        serializable object that contains the property 'predictions'. Our returned
        object therefore nests all the returned data within this 'predictions' 
        property.

        Args:
            approved (bool): Indicates if the loan has been approved.
            amount (float): The amount that has been approved.
            

        Returns:
            An object with one property 'predictions' that contains:
                success (bool): Always set to True. Indicate that the operation was successful.
                approved (bool): Indicates if the loan has been approved.
                amount (float): The loan amount that has been approved.
        """
        
        return {
            "predictions": [
                {
                    "success": True,
                    "approved": approved,
                    "amount": amount
                }
            ]
        }
    
    
    def _format_payload(fields, values):
        """Format the data to be sent to the deployed models.

        The data sent to the models deployed in CP4D must be formatted in a
        a JSON serializable object with a certain schema. The 'fields' property 
        is a list of all the variable names while the 'values' property is a 
        list of lists; each list contains N values (with N the number of fields). 

        Args:
            fields (str[]): A list of variable names.
            values (any[][]): A list of list of values.
            

        Returns:
            An object with one property 'input_data' that contains:
                fields (str[]): A list of variable names.
                values (any[][]): A list of list of values.
        """
        
        return {
            'input_data': [
                {
                    "fields": fields,
                    "values": values
                }
            ]
        }

            
    def score(payload):
        try:
            
            # Import all the dependencies
            import requests
            import pandas as pd
            import numpy as np
            from collections import namedtuple
            
            # Get the WML access token and setup the WML Client
            from ibm_watson_machine_learning import APIClient
            client = APIClient(params['wml_credentials'])
            space_uid = params['space_uid']
            client.set.default_space(space_uid)
            
            # Retrieve the input data
            fields = payload["input_data"][0]["fields"]
            values = payload["input_data"][0]["values"]
            
            if (DEBUG): print("Payload: ", payload)

            # Check that the parameters 'loan_amnt_applied' is given in the input data
            # Raise an error if not present.
            if "loan_amnt_applied" not in fields:
                raise ValueError("Missing parameters: loan_amnt_applied is missing from the fields property.")
 
            # Get the parameter 'loan_amnt_applied' from the payload
            value_idx = fields.index("loan_amnt_applied")
            loan_amnt_applied = values[value_idx]
  
            # Remove the parameter 'loan_amnt_applied' from the payload
            _ = fields.pop(value_idx)
            _ = values.pop(value_idx)
            
            if loan_amnt_applied <= min_requested_amnt:
                raise ValueError(f"Request amount too small: The requested amount must be greater than {min_requested_amnt}.")
                
            if loan_amnt_applied >= max_requested_amnt:
                raise ValueError(f"Request amount too big: The requested amount must be less than {max_requested_amnt}.")

            # -------------------------------------------------------
            # 1. score acceptance classifier
            # -------------------------------------------------------
            
            # Prepare the data to be sent to the model.
            job_payload_acceptance = _format_payload(fields, [values])
            if (DEBUG): print("1. scored acceptance classifier payload: ", job_payload_acceptance)

            # Score this data.
            job_details_acceptance = client.deployments.score(params["acceptance_uid"],job_payload_acceptance)
            if (DEBUG): print("1. scored acceptance classifier: ", job_details_acceptance)

            # Return if the loan has been rejected
            # Continue otherwise.
            if job_details_acceptance['predictions'][0]['values'][0][0] != approved_value:
                return _format_response(
                    approved=False
                )

            acceptance_pred_proba = job_details_acceptance['predictions'][0]['values'][0][1]
            if isinstance(acceptance_pred_proba, list):
                acceptance_pred_proba = acceptance_pred_proba[1]
 
            # -------------------------------------------------------
            # 2. score loan life classifier
            # -------------------------------------------------------
            
            # Create replicates of the customer information assuming different requested loan amounts,
            # spanning from a minimum amount of 'round_to' (see the configuration parameters of the function)
            # up to the original requested amount, with a step of 'round_to
            new_loan_amts = np.arange(loan_amnt_applied, 0, -round_to)
            new_loan_request_data = pd.DataFrame([values]  * len(new_loan_amts), columns=fields)
            new_loan_request_data["funded_amount"] = new_loan_amts

            # Prepare the data to be sent to the model
            values = new_loan_request_data.values.tolist()
            job_payload_life = _format_payload(fields, values)
            if (DEBUG): print("2. score loan life classifier payload: ",job_payload_life)
        
            # Score this data.
            job_details_life = client.deployments.score(params["life_uid"],job_payload_life)
            if (DEBUG): print("2. score loan life classifier score return: ", job_details_life)
            
            # Keep only the requests for which the requested amount has been accepted   
            Prediction = namedtuple('Prediction', ['resp', 'q', 'p'])
            predictions = []
            try:
                predictions = [Prediction(p[0], p[1][0], p[1][1]) for p in job_details_life['predictions'][0]['values']]
            except:
                predictions = [Prediction(p[0],0, p[1]) for p in job_details_life['predictions'][0]['values']]
          
            new_loan_request_data = new_loan_request_data.iloc[[idx for idx, pred in enumerate(predictions) if pred.resp == approved_value], :]
            
            # Save all the probabilities of acceptance for the requested that have been approved.
            life_pred_proba = [pred.p for pred in predictions if pred.resp == approved_value]

            # Return if none of the requests has been approved,
            # Continue otherwise.
            if len(life_pred_proba) == 0: 
                return _format_response(
                    approved=False
                )
            
            # -------------------------------------------------------
            # 3. score credit limit regressor
            # -------------------------------------------------------
            
            # Add the result of the first and second model to the data 
            # of all the loan requests that have been approved.
            
            new_loan_request_data['LoanStatus_proba'] = [acceptance_pred_proba] * len(life_pred_proba)
            new_loan_request_data['LifeLongerThanOneYear_proba'] = life_pred_proba
            new_loan_request_data.reset_index(drop=True, inplace=True)
            
            # Prepare the data to be sent to the model.
            values = new_loan_request_data.values.tolist()
            job_payload_credit = _format_payload(fields, values)
            if (DEBUG): print("3. score credit limit regressor payload: ", job_payload_credit)
            
            # Score this data.
            job_details_credit = client.deployments.score(params["credit_uid"],job_payload_credit)
            if (DEBUG): print("3. score credit limit regressor: ", job_details_credit)
            
            # Score credit limit regressor for a set of loan amounts
            predictions = job_details_credit['predictions'][0]['values']
            credit = [pred[0] for pred in predictions]
            
            # Return the maximum loan amount that has been approved.
            max_amt = float(new_loan_request_data.loc[np.argmax(credit), "funded_amount"])
            
            return _format_response(
                approved=True,
                amount=max_amt
            )
              
        except Exception as e:
            
            # Return the error in the CP4D-compliant format
            error_message = repr(e)
            
            if hasattr(e, 'message'):
                error_message = e.message

            return _format_error(400, error_message)
        
    return score

## Test the function locally

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# load a sample dataset
X_test = pd.read_csv(project.get_file('scoring-data.csv'))

# set of analytical quantities for target variables
analytics = ['LoanStatus',
             'LifeLongerThanOneYear',
             'Yield',
             'funded_amount',
             'loan_interest_rate'
            ]

# identify the features columns
features = [col for col in X_test.columns if col not in analytics]
X_test = X_test[features]

In [ ]:
# Prepare scoring payload.
job_payload = {
    "input_data": [{
        "fields": X_test.columns.tolist() + ["loan_amnt_applied"],
        "values": X_test.iloc[0, :].values.tolist() + [100000]
    }]
}

In [ ]:
score = optimal_credit_line()
score(job_payload)

## Deploy the function

In [ ]:
# Select Software specification to deploy the models 
sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")

In [ ]:
# Function Metadata.
meta_props = {
    client.repository.FunctionMetaNames.NAME: "credit_line_approval",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

In [ ]:
# Create the function artifact.

function_artifact = client.repository.store_function(meta_props=meta_props, function=optimal_credit_line)
function_uid = client.repository.get_function_uid(function_artifact)
print("Function UID = " + function_uid)

In [ ]:
# Details about the function.

function_details = client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

In [ ]:
# Display a list of all the functions.
client.repository.list_functions()

In [ ]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "online_credit_line_approval",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
# Create the deployment.
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)

In [ ]:
# Deployment UID.

deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

## Test the depoyed function

In [ ]:
# Prepare scoring payload.
job_payload = {
    "input_data": [{
        "fields": X_test.columns.tolist() + ["loan_amnt_applied"],
        "values": X_test.iloc[0, :].values.tolist() + [100000]
    }]
}

In [ ]:
# Perform prediction and display the result.

job_details = client.deployments.score(deployment_uid, job_payload)
job_details